In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait as ww
from bs4 import BeautifulSoup as bs
from collections import defaultdict
import time

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

base_url = "https://kidd.co.kr/news/list.php?mn=&c1=&c2=&c3=&key=&sch_date=&page={}"
article_base = "https://kidd.co.kr/"

dict = defaultdict(list)
for page in range(1, 2):
    url = base_url.format(page)
    driver.get(url)
    time.sleep(2)

    # 기사 제목과 링크 추출

    news_list = driver.find_elements(By.CSS_SELECTOR, 'div.list-news-items > a')

    print(news_list)
    for li in news_list:
        href = li.get_attribute('href')
        web_id = href.split('/')[-1]
        article_url = 'https://kidd.co.kr/news/' + web_id
        print(article_url)

        # open new window with article page
        driver.execute_script('window.open(arguments[0]);', article_url)
        driver.switch_to.window(driver.window_handles[1])

        article_soup = bs(driver.page_source, 'html.parser')

        active_category = article_soup.select_one('ul.kidd-nav-list > li.active > a')
        if active_category:
            category_name = active_category.get_text(strip=True)
            
        else:
            category_name = ''

        dict['Category'].append(category_name)



        dict['Title'].append(article_soup.select_one('#main > div > div > div.col-lg-8.col-xl-9 > div.news-content-main > div > div > div.articleTitleArea > div > h1').get_text(strip=True))
        
        date_element = article_soup.select_one('span.fs-5.text-muted')
        if date_element:
            full_text = date_element.get_text(strip=True)  # '기사입력 2025-11-27 19:46:50'
            date_part = full_text.split()[1]  # '2025-11-27'
            dict['Date'].append(date_part)
        else:
            dict['Date'].append('')  # 또는 '날짜 없음'

        sentences = []
        atc = article_soup.select_one('#act-newsbody > div.news-body-text.clearfix.text-break')
        p_tags = atc.find_all('p')[1:]

        for p_tag in p_tags:
            sentences.append(p_tag.get_text(strip = True))

        dict['Contents'].append(' '.join(sentences))


        driver.close()
        driver.switch_to.window(driver.window_handles[0])
driver.quit()

print(dict)

[<selenium.webdriver.remote.webelement.WebElement (session="fc662acc4882ccfd59a7e282439ebdaf", element="f.D3AABFF54DC7D49528B6CAB51A4A193E.d.EB45E021E978DC38D635F1E458371127.e.379")>, <selenium.webdriver.remote.webelement.WebElement (session="fc662acc4882ccfd59a7e282439ebdaf", element="f.D3AABFF54DC7D49528B6CAB51A4A193E.d.EB45E021E978DC38D635F1E458371127.e.387")>, <selenium.webdriver.remote.webelement.WebElement (session="fc662acc4882ccfd59a7e282439ebdaf", element="f.D3AABFF54DC7D49528B6CAB51A4A193E.d.EB45E021E978DC38D635F1E458371127.e.395")>, <selenium.webdriver.remote.webelement.WebElement (session="fc662acc4882ccfd59a7e282439ebdaf", element="f.D3AABFF54DC7D49528B6CAB51A4A193E.d.EB45E021E978DC38D635F1E458371127.e.403")>, <selenium.webdriver.remote.webelement.WebElement (session="fc662acc4882ccfd59a7e282439ebdaf", element="f.D3AABFF54DC7D49528B6CAB51A4A193E.d.EB45E021E978DC38D635F1E458371127.e.411")>, <selenium.webdriver.remote.webelement.WebElement (session="fc662acc4882ccfd59a7e2824

In [24]:
import json

articles = []
for i in range(len(dict['Category'])):
    article = {
        "num": i+1,
        "Category": dict['Category'][i],
        "Date": dict['Date'][i],
        "Title": dict['Title'][i],
        "Contents": dict['Contents'][i]
    }
    articles.append(article)

# JSON으로 저장
with open('articles_kidd.json', 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)
